In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML
import numpy as np
from scipy.constants import physical_constants

In [ ]:
def get_ani(traj,dx,dy,t,frames=None):#(xdata,ydata,xlim,ylim,frames=None):
    
    xdata,ydata,cdata,xlim,ylim=traj_coord(traj,dx,dy)
    if frames is None:
        frames=len(ydata)
    
    fig, ax = plt.subplots()
    ax.set_xlim((xlim))
    ax.set_ylim((ylim))
    l = ax.scatter([],[])
    
    def animate(i):
        l.set_offsets(np.stack((xdata[i],ydata[i]),axis=1))
        l.set_color(cdata[i])
        ax.set_title('Time: {:.3E} [s]'.format(t[i]))
    
    ani = matplotlib.animation.FuncAnimation(fig, animate, frames=frames)
    plt.close()
    
    return ani

In [ ]:
def calc_full_traj(traj,trajtobeupdated):
    trajfull=[]
    trajfull.append(traj[0].copy())
    for i in range(1,len(traj)):
        trajfull.append(trajfull[i-1].copy())
        k=0
        for j in trajtobeupdated[i]:
            trajfull[i][j]=traj[i][k]
            k+=1
    return trajfull

In [ ]:
def traj_coord(traj,dx,dy):
    X=[]
    Y=[]
    C=[]
    Xmax=-np.inf
    Ymax=-np.inf
    for i in range(len(traj)):
        x,y,c=coordinates(traj[i],dx,dy)
        xmax=max(x) 
        Xmax=xmax if xmax>Xmax else Xmax
        ymax=max(y) 
        Ymax=ymax if ymax>Ymax else Ymax
        X.append(x.copy())
        Y.append(y.copy())
        C.append(c.copy())
    xlim=(-10,Xmax+10)
    ylim=(-20,Ymax+20)
    return X,Y,C,xlim,ylim

## Import kinetic_monte_carlo.py functions

> coordinates <br>
neighbors <br>
events <br>
pbc <br>
....

In [ ]:
from kinetic_monte_carlo import *

## Tasks

1. Execute the program with the following parameters: 

 * update graph each steps 1000 
 * coverage 0.3
 * number of steps 130000
 * temperature in K 300
 * diffusion barrier 0.1
 * binding barrier 0.1

   Observe how events occur. Observe how time evolves. Did the job perform all the 300000 steps? Why? Observe the patterns obtained.


2. Execute the program with the following parameters: 

  * update graph each steps 1000
  * coverage 0.3
  * number of steps 300000
  * temperature in K 300
  * diffusion barrier 0.3
  * binding barrier 0.4
  
    Do you notice differences in the way events occur? How is evolving time compared to the previous case? How does the final geometry differ from the previous case? 
    
    
3. At each step of the simulaiton the list of possible events is created (or, better, updated). An event is chosen randomly and is then actuated. Would it be possible to execute simultaneously more events at each KMC step? 

## Kinetic Monte Carlo parameters

In [ ]:
kb=physical_constants['Boltzmann constant in eV/K'][0]
nsteps=150000      #number of steps
T=300              #temperature [K]
db=0.1             #diffusion barrier
bb=0.1             #binding barrier
dd=db/20
beta=1.0/(kb*T)
gamma_d=np.float64(10**10)
gamma_b=np.float64(10**8)

## Honeycomb lattice define

In [ ]:
nx=30
ny=nx/2
dx=13.5
sr3=np.sqrt(3.0)
dy=dx*sr3

<img src="hex_convention.png" width=500 height=500>

The figure illustrates the hexagonal lattice. The blue dot (1) is positioned at site $si=0$ of cell [$xi$,$yi$]=[$4,2$] and the blue dot (5) at site $si=1$ of the same cell.

<img src="rates_explain.png" width=500 height=500>

The figure illustrates shemes to summarise the different possible diffusion events

## #Molecules

In [ ]:
coverage=0.2#float(raw_input("coverage "))
nmolecules=int(nx*nx*coverage)

> ## **Rates**

In [ ]:
rates={
    # free diffusion
'd' : gamma_d * np.exp(-beta*(db)) ,  
    # 'dri' means that I am moivg to site i (relative to my position) remaining 
    #  attached to at least one of my neighbors.
'dr1': gamma_d * np.exp(-beta*(db)) , 
'dr2': gamma_d * np.exp(-beta*(db+dd)) , 
'drl2': gamma_d * np.exp(-beta*(db+1.5*dd)) , 
'dr3': gamma_d * np.exp(-beta*(db+3.0*dd)) , 
'dr4': gamma_d * np.exp(-beta*(db+3.0*dd)) , 
'dr5': gamma_d * np.exp(-beta*(db+6.0*dd)) , 
    # 'di' means that I am moving in a site which is not neighbooring to any 
    # of my neighbors.
'd1': gamma_d * np.exp(-beta*(db+2*dd)) ,  # d1 moving away having 1nb
'd2': gamma_d * np.exp(-beta*(db+4.0*dd)) ,  # d2 moving away having 2nb
'd3': gamma_d * np.exp(-beta*(db+4*dd)) ,  # d3 moving away having 3nb
'b':  gamma_b * np.exp(-beta*bb)\
}

In [ ]:
### compute the total rate of events
def total_rate(events,rates):
    '''
        TODO: compute the total rate R
    '''
    return R

### select randomly an event from the list of possible events
def find_event(R,rates,events):
    '''
        TODO: given the total rate R, the rates and the events, 
              select and return the_event
    '''
    return the_event

### evolve time according to Poisson distribution
def evolve_time(t,R):
    '''
        TODO: evolve the time according to Poisson's statistics.
    '''
    return t

## Run ...

In [ ]:
molecules=[]
i=0

#### Create initial geometry
while i < nmolecules :
    xi=np.random.randint(0,nx)
    yi=np.random.randint(0,ny)
    si=np.random.randint(0,2)
    newmolecule=[xi,yi,si,0]
    if  newmolecule  not in molecules:
        molecules.append(newmolecule)
        i=i+1
#### END Create initial geometry

#### MAIN KMC LOOP
t=np.zeros(nsteps+1)

#### at the beginning we have to check possible events for all molecules
tobeupdated=[iu for iu in range(len(molecules))]
possible_events=[]

traj=[]
time=[]
traj.append(molecules.copy())
time.append(0.0)
steps_per_frame=1000

for i in range(nsteps):

    ### create the list of possible events
    possible_events=possible_events+events(molecules,tobeupdated,nx,ny)
    if possible_events==[]:
        print("no more events possible") #DIFF. from MC (which continues 2 inf.)
        traj.append(molecules.copy())
        time.append(t[i])
        break

    ### compute the total rate of events 
    R=total_rate(possible_events,rates) # e.g. [[0,0,1,0]]

    ### select randomly an event
    selected_event=find_event(R,rates,possible_events)

    ### apply the event
    possible_events,tobeupdated=apply_event(molecules,selected_event,possible_events,nx,ny)

    ### evolve time according to Poisson distribution
    t[i+1]=evolve_time(t[i],R)
    
    if (i%steps_per_frame)==0:
        traj.append(molecules.copy())
        time.append(t[i])

## Create animation

Visualize the trajectory.

In [ ]:
ani = get_ani(traj,dx,dy,time)

In [ ]:
HTML(ani.to_jshtml())